In [1]:
%%writefile mapper.py


import sys
import re


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

path = 'stop_words_en.txt'

# Your code for reading stop words here

with open(path, "r") as f:
    stop_words = f.read().splitlines()

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue

    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        
    # Your code for mapper here.
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        if word.lower() in stop_words:
            print >> sys.stderr, "reporter:counter:Wiki stats,Stop words,%d" % 1
        print("{}\t{}".format(word, 1))

Overwriting mapper.py


In [2]:
%%writefile reducer.py

# Your code for reducer here.
import sys

current_key = None
total_words = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)

        if current_key != key:
            
            if current_key:
                print("{}\t{}".format(current_key, total_words))
            
            total_words = 0
            current_key = key
            
        total_words += count
    
    except ValueError as e:
        continue
    
if current_key:
    print("{}\t{}".format(current_key, total_words))

Overwriting reducer.py


In [3]:
%%writefile counter_process.py

#! /usr/bin/env python

import sys

# Your functions may be here.


stop_words = 0
total_words = 1

if __name__ == '__main__':
    # Your code here.
    
    for line in sys.stdin:
        try: 
            key, value = line.strip().split('=', 1)
            
            if key == sys.argv[1]: 
                stop_words = float(value)
            elif key == sys.argv[2]: 
                total_words = float(value)
        
        except Exception as e:
            continue
            
    print((stop_words / total_words) * 100)

Writing counter_process.py


In [4]:
%%bash

OUT_DIR="coursera_mr_task2"$(date +"%s%6N")
NUM_REDUCERS=8
LOGS="stderr_logs.txt"

# Stub code for your job

rm ${LOGS} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming stop words" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> $LOGS
 
cat $LOGS | python ./counter_process.py "Stop words" "Total words"
cat $LOGS >&2

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

41.60262201698447


rm: cannot remove 'stderr_logs.txt': No such file or directory
19/06/15 15:07:00 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/06/15 15:07:01 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/06/15 15:07:02 INFO mapred.FileInputFormat: Total input files to process : 1
19/06/15 15:07:02 INFO mapreduce.JobSubmitter: number of splits:2
19/06/15 15:07:02 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1560606164948_0002
19/06/15 15:07:03 INFO impl.YarnClientImpl: Submitted application application_1560606164948_0002
19/06/15 15:07:03 INFO mapreduce.Job: The url to track the job: http://307727ece75b:8088/proxy/application_1560606164948_0002/
19/06/15 15:07:03 INFO mapreduce.Job: Running job: job_1560606164948_0002
19/06/15 15:07:12 INFO mapreduce.Job: Job job_1560606164948_0002 running in uber mode : false
19/06/15 15:07:12 INFO mapreduce.Job:  map 0% reduce 0%
19/06/15 15:07:31 INFO mapreduce.Job:  map 2% reduce 0%
19/06/15 15:07:37 